In [1]:
%pwd

'/home/yogesharyal/Documents/e2e_kidney_stone_detection/research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/yogesharyal/Documents/e2e_kidney_stone_detection'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Aryog/Kidney_classification_using_MLFLOW.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Aryog" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="dbc1fd14e2ede1e244dd48c1531e2477cf138b25"

export MLFLOW_TRACKING_URI=https://dagshub.com/Aryog/Kidney_classification_using_MLFLOW.mlflow   
export MLFLOW_TRACKING_USERNAME=Aryog   
export MLFLOW_TRACKING_PASSWORD=dbc1fd14e2ede1e244dd48c1531e2477cf138b25  

In [6]:
import tensorflow as tf


2024-05-18 20:20:03.811903: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-18 20:20:03.812128: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-18 20:20:03.814743: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-18 20:20:03.854092: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 20:20:04.607244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from kidneyClassifier.constants import *
from kidneyClassifier.utils.common import read_yaml,create_directories,save_json
 

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Dataset/Train",
            mlflow_uri="https://dagshub.com/Aryog/Kidney_classification_using_MLFLOW.mlflow",
            all_params=self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.3
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model= self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        # mlflow.set_registry_url(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.log_params(self.config.all_params)
        mlflow.log_metrics(
            {"loss": self.score[0], "accuracy": self.score[1]}
        )

        # Model registry does not work with file store
        if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the model registry, which depends upon the usage
            mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")
        else:
            mlflow.kears.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 435 images belonging to 2 classes.


/home/yogesharyal/.conda/envs/mlproj/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


28/28 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.5022 - loss: 4.1582


2024/05/18 20:20:58 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/05/18 20:21:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
